https://github.com/ziyujia/GraphSleepNet


In [ ]:
# Generic
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

# Misc
from scipy.sparse.linalg import eigs
from scipy.spatial import distance_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import KFold

# Tensorflow
import keras
import tensorflow as tf
from keras import backend as K
from keras import layers
from keras import regularizers
from keras import models
from keras.layers import Layer
from keras.layers.core import Lambda
from tensorflow.keras.layers import LayerNormalization
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

Num GPUs Available:  1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load Data

In [ ]:
'''
cov_idx_dict = {'lab': (1,6),
               'session': (6,10),
               'x': (10,11),
               'y': (11,12),
               'z': (12,13),
               'waveform amplitude': (13,14),
               'waveform width': (14,15),
               'paw speed': (15,16),
               'nose speed': (16,17),
               'pupil diameter': (17,18),
               'motion energy': (18,19),
               'stimuli': (19,21),
               'go cue': (21,22),
               'choice': (22,24),
               'reward': (24,26),
               'wheel velocity': (26,27),
               'mouse prior': (27,28),
               'lick': (28,29),
               'decision strategy (GLM-HMM)': (29,33),
               'brain region': (33,38),
               'noise': (38,39),
               'all': (1,39)}
''';

In [ ]:
path = "drive/MyDrive/Neural/ibl/"

firing_rates = np.load("".join([path, "unsorted/unsorted_firing_rates.npy"]))
firing_rates = firing_rates.transpose([1,0,2])
n_channels, n_trials, n_time_bins = firing_rates.shape

#aggregate_amps = np.load("".join([path, "unsorted_aggregate_amps.npy"]))
#aggregate_amps = aggregate_amps.transpose([1,0,2])

neural_feature = np.load("".join([path, "sorted/d23a44ef-1402-4ed7-97f5-47e9a7a504d9_feature.npy"]))
decisions = neural_feature[:,:,:,22:24].sum(2)[0,:,:]

probe_geometry = np.load("".join([path, "np1_channel_map.npy"]))

unsorted_fire_rates = firing_rates.transpose([1,2,0]).reshape(-1, n_time_bins*n_channels)
for k in range(n_time_bins):
  unsorted_fire_rates[:, k*n_channels:(k+1)*n_channels] = (unsorted_fire_rates[:, k*n_channels:(k+1)*n_channels] - unsorted_fire_rates[:, k*n_channels:(k+1)*n_channels].mean()) / unsorted_fire_rates[:, k*n_channels:(k+1)*n_channels].std()

#resorted_aggregate_amps = aggregate_amps.transpose([1,2,0]).reshape(-1, n_time_bins*n_channels)
#for k in range(n_time_bins):
#  resorted_aggregate_amps[:, k*n_channels:(k+1)*n_channels] = (resorted_aggregate_amps[:, k*n_channels:(k+1)*n_channels] - resorted_aggregate_amps[:, k*n_channels:(k+1)*n_channels].mean()) / resorted_aggregate_amps[:, k*n_channels:(k+1)*n_channels].std()


#Xs = np.concatenate(
#      [resorted_fire_rates.reshape(-1, n_time_bins, n_channels)[:,:,:,np.newaxis],
#       resorted_aggregate_amps.reshape(-1, n_time_bins, n_channels)[:,:,:,np.newaxis]], 3
#)
Xs = unsorted_fire_rates.reshape(-1, n_time_bins, n_channels)[:,:,:,np.newaxis]
ys = decisions
print("Number of channels is", n_channels)
print("Number of trials is", n_trials)
print("Number of time bins is", n_time_bins)

Number of channels is 384
Number of trials is 286
Number of time bins is 30


Find brain ROIs of each channel:

In [ ]:
# Load brain regions by channel and depth data
#ROI_by_channel_depth = np.load('drive/MyDrive/ibl/ROI_by_channel_depth.npy')
#channels = ROI_by_channel_depth[:,0].astype(int)
#depths = ROI_by_channel_depth[:,1].astype(float)
#ROIs = ROI_by_channel_depth[:,2]

In [ ]:
# Match channels to brain regions by channel index and depth
#brain_regions = []
#for i in np.arange(n_channels):
#  region = ROIs[np.where(channels == i)]
#  if len(region) == 0:
#    region = (ROIs[np.where(depths == probe_geometry[i,1])])
#  if len(region) == 0:
#    region = ['NA']
#  region = region[0]
#  if (region == 'ml') or (region == 'cing') or (region == 'or'):
#    region = 'NA'
#  brain_regions.append(region)

In [ ]:
# Convert categorical data to integer-valued array
#brain_region_dict = {'NA':0, 'CA1':1, 'DG':2, 'Eth':3, 'LP':4, 'PO':5, 'PPC':6, 'TH':7, 'VPLpc':8, 'alv':9, 'fp':10}
#brain_regions = [brain_region_dict[i] for i in brain_regions]

ST-GCN

In [ ]:
def calc_rescaled_graph_Laplacian(A):
    '''
    Compute \tilde{L} = (2 / \lambda_max) * L - I_M, 
            L = D - A, D is degree matrix of A, 
            \lambda_max is the largest eigenvalue of L
    Input:
    ----------
    A: (M, M) adjacency matrix, M is the num of vertices (neurons / channels)

    Output:
    ----------
    L_tilde: (M, M) rescaled graph Laplacian
    '''
    assert A.shape[0] == A.shape[1]
    M = A.shape[0]   
    # Compute the degree matrix of A
    D = np.diag(np.sum(A, axis=1))
    # Compute the graph Laplacian
    L = D - A   
    # Find the largest real eigenvalues 
    lambda_max = (np.linalg.eig(L)[0])[0]   
    # Compute the rescaled graph Laplacian
    L_tilde = (2 / lambda_max) * L - np.identity(M)
    
    return L_tilde


def calc_cheb_polynomial(L_tilde, K):
    '''
    Recursively compute K chebyshev polynomials from T_0 to T_{K-1}, T_0 = 1, T_1 = x
    Input:
    ----------
    L_tilde: (M, M) rescaled graph Laplacian
    K: Order of chebyshev polynomials

    Output:
    ----------
    cheb_poly: A list of K chebyshev polynomials
    '''
    M = L_tilde.shape[0]

    # Recursively compute chebyshev polynomials
    cheb_poly = [np.identity(M), L_tilde.copy()]
    for i in range(2, K):
        cheb_poly.append(2 * L_tilde * cheb_poly[i - 1] - cheb_poly[i - 2]) 
    
    return cheb_poly

In [ ]:
class Temporal_Attention_Layer(Layer):
    '''
    Compute temporal attention scores.
    --------
    Input:  (n_trials, n_time_bins, n_nodes, n_features)
    Output: (n_trials, n_time_bins, n_time_bins)
    '''
    def __init__(self, **kwargs):
        super(Temporal_Attention_Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        _, n_time_bins, n_nodes, n_features = input_shape

        self.U1 = self.add_weight(name='U1', shape=(n_nodes, 1), initializer='uniform', trainable=True)
        self.U2 = self.add_weight(name='U2', shape=(n_features, n_nodes), initializer='uniform', trainable=True)
        self.U3 = self.add_weight(name='U3', shape=(n_features, ), initializer='uniform', trainable=True)
        self.be = self.add_weight(name='be', shape=(1, n_time_bins, n_time_bins), initializer='uniform', trainable=True)
        self.Ve = self.add_weight(name='Ve', shape=(n_time_bins, n_time_bins), initializer='uniform', trainable=True)
        super(Temporal_Attention_Layer, self).build(input_shape)

    def call(self, x):
        _, n_time_bins, n_nodes, n_features = x.shape
        
        # (n_trials, n_time_bins, n_nodes)
        lhs = K.dot(tf.transpose(x, perm=[0,1,3,2]), self.U1)
        lhs = tf.reshape(lhs, [tf.shape(x)[0], n_time_bins, n_features])
        lhs = K.dot(lhs, self.U2)
        
        # (n_trials, n_nodes, n_time_bins)
        rhs = K.dot(self.U3, tf.transpose(x, perm=[2,0,3,1])) 
        rhs = tf.transpose(rhs, perm=[1,0,2])
        
        # (n_trials, n_time_bins, n_time_bins)
        product = K.batch_dot(lhs, rhs)
        
        S = tf.transpose(K.dot(self.Ve, tf.transpose(K.sigmoid(product + self.be), perm=[1, 2, 0])), perm=[2, 0, 1])
        
        # Normalization
        S = S - K.max(S, axis = 1, keepdims = True)
        exp = K.exp(S)
        S_normalized = exp / K.sum(exp, axis = 1, keepdims = True)
        return S_normalized

In [ ]:
def reshape_dot(x):
  '''
  Apply temporal attention to x.
  Input:  [x, temporal_attention_scores]
  '''
  x, temporal_attention_scores = x
  return tf.reshape(
      K.batch_dot(
          tf.reshape(tf.transpose(x, perm = [0,2,3,1]), (tf.shape(x)[0], -1, tf.shape(x)[1])), 
          temporal_attention_scores),
      [-1, x.shape[1],x.shape[2],x.shape[3]])

In [ ]:
class Spatial_Attention_Layer(Layer):
    '''
    Compute spatial attention scores.
    --------
    Input:  (n_trials, n_time_bins, n_nodes, n_features)
    Output: (n_trials, n_nodes, n_nodes)
    '''
    def __init__(self, **kwargs):
        super(Spatial_Attention_Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        _, n_time_bins, n_nodes, n_features = input_shape
        self.W1 = self.add_weight(name='W1', shape=(n_time_bins, 1), initializer='uniform', trainable=True)
        self.W2 = self.add_weight(name='W2', shape=(n_features, n_time_bins), initializer='uniform', trainable=True)
        self.W3 = self.add_weight(name='W3', shape=(n_features, ), initializer='uniform', trainable=True)
        self.bs = self.add_weight(name='bs', shape=(1, n_nodes, n_nodes), initializer='uniform', trainable=True)
        self.Vs = self.add_weight(name='Vs', shape=(n_nodes, n_nodes), initializer='uniform', trainable=True)
        super(Spatial_Attention_Layer, self).build(input_shape)

    def call(self, x):
        _, n_time_bins, n_nodes, n_features = x.shape
        
        # (n_trials, n_nodes, n_time_bins)
        lhs = K.dot(tf.transpose(x, perm = [0,2,3,1]), self.W1)
        lhs = tf.reshape(lhs, [tf.shape(x)[0], n_nodes, n_features])
        lhs = K.dot(lhs, self.W2)
        
        # (n_trials, n_time_bins, n_nodes)
        rhs = K.dot(self.W3, tf.transpose(x, perm = [1,0,3,2])) 
        rhs = tf.transpose(rhs, perm = [1,0,2])
        
        # (n_trials, n_nodes, n_nodes)
        product = K.batch_dot(lhs, rhs)
        
        S = tf.transpose(K.dot(self.Vs, tf.transpose(K.sigmoid(product + self.bs), perm = [1, 2, 0])), perm = [2, 0, 1])
        
        # Normalization
        S = S - K.max(S, axis = 1, keepdims = True)
        exp = K.exp(S)
        S_normalized = exp / K.sum(exp, axis = 1, keepdims = True)
        return S_normalized

In [ ]:
class Spatial_Conv_with_Spatial_Attention(Layer):
    '''
    K-order spatial graph convolution.
    --------
    Input:  - x   (n_trials, n_time_bins, n_nodes, n_features)
            - spatial_attention_scores (n_trials, n_nodes, n_nodes)
    Output: (n_trials, n_time_bins, n_nodes, n_filters)
    '''
    def __init__(self, n_filters, k, cheb_poly, **kwargs):
        self.k = k
        self.n_filters = n_filters
        self.cheb_poly = cheb_poly
        super(Spatial_Conv_with_Spatial_Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        x_shape, spatial_attention_scores_shape = input_shape
        _, n_time_bins, n_nodes, n_features = x_shape
        self.Theta = self.add_weight(name='Theta', shape=(self.k, n_features, self.n_filters), initializer='uniform', trainable=True)
        super(Spatial_Conv_with_Spatial_Attention, self).build(input_shape)

    def call(self, x):
        assert isinstance(x, list)
        assert len(x)==2 
        x, spatial_attention_scores = x
        _, n_time_bins, n_nodes, n_features = x.shape
        
        outputs=[]
        for time_bin in range(n_time_bins):
            # (n_trials, n_nodes, n_features)
            graph_signal = x[:, time_bin, :, :]
            # (n_trials, n_nodes, n_filters)
            output = tf.zeros(shape = (tf.shape(x)[0], n_nodes, self.n_filters))
            
            for k in range(self.k):
                # (n_nodes, n_nodes)
                T_k = self.cheb_poly[k]
                    
                # (n_trials, n_nodes, n_nodes)
                T_k_with_AT = T_k * spatial_attention_scores

                # (n_features, n_filters)
                theta_k = self.Theta[k]

                # shape is (n_trials, n_nodes, n_features)
                rhs = K.batch_dot(tf.transpose(T_k_with_AT, perm = [0, 2, 1]), graph_signal)

                output = output + K.dot(rhs, theta_k)
            outputs.append(tf.expand_dims(output,1))
            
        return K.relu(K.concatenate(outputs, axis = 1))

In [ ]:
def Spatial_Temporal_Convolution_Block(x, k, n_spatial_filters, n_time_filters, cheb_poly, time_conv_strides, time_conv_kernel, i):
    '''
    Spatial-Temporal Convolution Block.
    -------
    x: (n_trials, n_time_bins, n_nodes, n_features)
    '''     
    # Apply Temporal Attention (n_trials, n_time_bins, n_time_bins)
    temporal_attention_scores = Temporal_Attention_Layer()(x)
    x_temporal_attention = Lambda(reshape_dot, name = 'reshape_dot' + str(i))([x, temporal_attention_scores])

    # Apply Spatial Attention (n_trials, n_nodes, n_nodes)
    spatial_attention = Spatial_Attention_Layer()(x_temporal_attention)
    
    # Apply Spatial Graph Convolution with Chebyshev polynomial expansions (n_trials, n_time_bins, n_nodes, n_spatial_filters)
    spatial_conv_output = Spatial_Conv_with_Spatial_Attention(n_spatial_filters, k, cheb_poly)([x, spatial_attention])
    
    # Apply Temporal Convolution (n_trials, n_time_bins, n_nodes, n_time_filters)
    time_conv_output = layers.Conv2D(
        filters = n_time_filters, 
        kernel_size = (time_conv_kernel, 1), 
        padding = 'same', 
        strides = (time_conv_strides, 1)
    )(spatial_conv_output)

    # Apply Residual Shortcut (n_trials, n_time_bins, n_nodes, n_time_filters)
    x_residual = layers.Conv2D(
        filters = n_time_filters, 
        kernel_size = (1, 1), 
        strides = (1, time_conv_strides)
    )(x)
    
    # Layer Normalization
    layer_norm = LayerNormalization(name = None, axis = 3, epsilon = 1e-8, dtype = tf.float32)
    relu_x = K.relu(x_residual + time_conv_output)
    output = layer_norm(relu_x)
    return output

In [ ]:
def build_STGCN(k, data_shape, n_spatial_filters, n_time_filters, cheb_poly, time_conv_strides, time_conv_kernel, 
                n_blocks, hidden_sizes, opt, regularizer, drop_out):
    
    # Input:  (*, n_time_bins, n_nodes, n_features)
    data_layer = layers.Input(shape = data_shape, name = 'Input Data')
    
    # Spatial Temporal Convolution Block
    output = Spatial_Temporal_Convolution_Block(data_layer, k, n_spatial_filters, n_time_filters, cheb_poly, time_conv_strides, time_conv_kernel, 0)
    for i in range(1, n_blocks):
        output = Spatial_Temporal_Convolution_Block(output, k, n_spatial_filters, n_time_filters, cheb_poly, time_conv_strides, time_conv_kernel, i)
   
    # Global Dense Layers
    output = layers.Flatten()(output)
    for size in hidden_sizes:
        output = layers.Dense(size)(output)
    
    # Drop Out
    if drop_out!=0:
        output = layers.Dropout(drop_out)(output)
    
    # Classifier
    output = layers.Dense(2, activation = 'softmax', kernel_regularizer = regularizer)(output)
    
    model = models.Model(inputs = data_layer, outputs = output)
    
    METRICS = [keras.metrics.Accuracy(name="accuracy"), keras.metrics.AUC(name='auc')]

    model.compile(
        optimizer = opt,
        loss = 'categorical_crossentropy',
        metrics = METRICS,
    )
    return model

Train

In [ ]:
n_epochs   = 50
batch_size = 15
optimizer  = "SGD"
learn_rate = 1e-3
lr_decay   = 2e-6

n_features            = 1
data_shape            = (n_time_bins, n_channels, n_features)
K_order               = 3
n_blocks              = 1
n_spatial_filters     = 10 
n_time_filters        = 10 
time_conv_strides     = 1
time_conv_kernel      = 3 
hidden_sizes          = np.array([64, 32], dtype=int)
drop_out              = 0 
l1_penalty            = 0.01  
l2_penalty            = 0.01  

if optimizer == "adam":
    opt = tf.keras.optimizers.Adam(learning_rate = learn_rate, decay = lr_decay, clipvalue = .5) 
elif optimizer == "SGD":
    opt = tf.keras.optimizers.SGD(learning_rate = learn_rate, decay = lr_decay, clipvalue = .5) 

if l1_penalty != 0 and l2_penalty != 0:
    regularizer = keras.regularizers.l1_l2(l1 = l1_penalty, l2 = l2_penalty)
elif l1_penalty != 0 and l2_penalty == 0:
    regularizer = keras.regularizers.l1(l1_penalty)
elif l1_penalty == 0 and l2_penalty != 0:
    regularizer = keras.regularizers.l2(l2_penalty)
else:
    regularizer = None

In [ ]:
# Create the adjacency matrix
A = np.ones((n_channels, n_channels))
#A = np.zeros((n_channels, n_channels))
#for i in range(n_channels):
#  for j in range(n_channels):
#    if brain_regions[i] == brain_regions[j]:
#      A[i, j] = 1

In [ ]:
# Compute Chebyshev polynomials
L = calc_rescaled_graph_Laplacian(A)
cheb_poly = calc_cheb_polynomial(L, K_order)

K-Fold CV

In [ ]:
kfold = KFold(n_splits = 10, shuffle = True, random_state = seed) 

fold = 0
cv_accs = []
cv_aucs = []
for train, test in kfold.split(Xs, ys):

  model = build_STGCN(
    K_order, data_shape, n_spatial_filters, n_time_filters, cheb_poly, time_conv_strides, time_conv_kernel, 
    n_blocks, hidden_sizes, opt, regularizer, drop_out)
  
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold} ...')

  model.fit(Xs[train], ys[train],
        epochs = n_epochs,
        batch_size = batch_size,
        shuffle = True,
        verbose = 1)
  
  test_probs = model.predict(Xs[test])
  test_preds = test_probs.argmax(1)
  acc = accuracy_score(ys[test].argmax(1), test_preds)
  auc = roc_auc_score(ys[test], test_probs)
  cv_accs.append(acc)
  cv_aucs.append(auc)
  print(f'Test accuracy: {acc}, AUC: {auc} .')
  fold += 1

print(f'Average {fold}-fold CV accuracy: {np.mean(cv_accs)}, AUC: {np.mean(cv_aucs)} .')
print(f'SD of {fold}-fold CV accuracy: {np.std(cv_accs)}, AUC: {np.std(cv_aucs)} .')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/tensor_util.py:436: ComplexWarning: Casting complex values to real discards the imaginary part
  nparray = values.astype(dtype.as_numpy_dtype)


------------------------------------------------------------------------
Training for fold 0 ...
Epoch 1/50
18/18 [==============================] - 25s 52ms/step - loss: 41.7058 - accuracy: 0.4319 - auc: 0.6042
Epoch 2/50
18/18 [==============================] - 1s 48ms/step - loss: 20.9430 - accuracy: 0.4942 - auc: 0.7038
Epoch 3/50
18/18 [==============================] - 1s 48ms/step - loss: 17.6254 - accuracy: 0.4455 - auc: 0.6307
Epoch 4/50
18/18 [==============================] - 1s 48ms/step - loss: 6.1386 - accuracy: 0.3463 - auc: 0.8212
Epoch 5/50
18/18 [==============================] - 1s 48ms/step - loss: 4.4223 - accuracy: 0.3794 - auc: 0.8700
Epoch 6/50
18/18 [==============================] - 1s 48ms/step - loss: 5.5227 - accuracy: 0.3696 - auc: 0.7610
Epoch 7/50
18/18 [==============================] - 1s 49ms/step - loss: 0.8437 - accuracy: 0.2821 - auc: 0.9608
Epoch 8/50
18/18 [==============================] - 1s 48ms/step - loss: 3.3695 - accuracy: 0.2938 - auc: 0.

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/tensor_util.py:436: ComplexWarning: Casting complex values to real discards the imaginary part
  nparray = values.astype(dtype.as_numpy_dtype)


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
18/18 [==============================] - 8s 53ms/step - loss: 26.9840 - accuracy: 0.4572 - auc: 0.6089
Epoch 2/50
18/18 [==============================] - 1s 50ms/step - loss: 13.7564 - accuracy: 0.2918 - auc: 0.6991
Epoch 3/50
18/18 [==============================] - 1s 51ms/step - loss: 4.5008 - accuracy: 0.1946 - auc: 0.8501
Epoch 4/50
18/18 [==============================] - 1s 50ms/step - loss: 0.3850 - accuracy: 0.0817 - auc: 0.9846
Epoch 5/50
18/18 [==============================] - 1s 51ms/step - loss: 1.4942 - accuracy: 0.0467 - auc: 0.9137
Epoch 6/50
18/18 [==============================] - 1s 51ms/step - loss: 3.7878 - accuracy: 0.2957 - auc: 0.7973
Epoch 7/50
18/18 [==============================] - 1s 50ms/step - loss: 0.1714 - accuracy: 0.0331 - auc: 1.0000
Epoch 8/50
18/18 [==============================] - 1s 51ms/step - loss: 0.1684 - accuracy: 0.0331 - auc: 1.00

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/tensor_util.py:436: ComplexWarning: Casting complex values to real discards the imaginary part
  nparray = values.astype(dtype.as_numpy_dtype)


...

------------------------------------------------------------------------
Training for fold 7 ...
Epoch 1/50
18/18 [==============================] - 9s 52ms/step - loss: 35.0436 - accuracy: 0.4729 - auc: 0.6250
Epoch 2/50
18/18 [==============================] - 1s 49ms/step - loss: 29.4999 - accuracy: 0.4554 - auc: 0.6473
Epoch 3/50
18/18 [==============================] - 1s 49ms/step - loss: 12.9205 - accuracy: 0.4864 - auc: 0.7134
Epoch 4/50
18/18 [==============================] - 1s 50ms/step - loss: 10.0911 - accuracy: 0.3740 - auc: 0.7388
Epoch 5/50
18/18 [==============================] - 1s 50ms/step - loss: 15.1942 - accuracy: 0.4147 - auc: 0.6068
Epoch 6/50
18/18 [==============================] - 1s 49ms/step - loss: 6.9466 - accuracy: 0.3547 - auc: 0.8021
Epoch 7/50
18/18 [==============================] - 1s 49ms/step - loss: 3.7555 - accuracy: 0.4109 - auc: 0.8881
Epoch 8/50
18/18 [==============================] - 1s 50ms/step - loss: 1.5009 - accuracy: 0.3547 - auc: 0

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/tensor_util.py:436: ComplexWarning: Casting complex values to real discards the imaginary part
  nparray = values.astype(dtype.as_numpy_dtype)


------------------------------------------------------------------------
Training for fold 8 ...
Epoch 1/50
18/18 [==============================] - 9s 53ms/step - loss: 39.6410 - accuracy: 0.4283 - auc: 0.5936
Epoch 2/50
18/18 [==============================] - 1s 50ms/step - loss: 28.1360 - accuracy: 0.4864 - auc: 0.6390
Epoch 3/50
18/18 [==============================] - 1s 50ms/step - loss: 17.9143 - accuracy: 0.4477 - auc: 0.6715
Epoch 4/50
18/18 [==============================] - 1s 50ms/step - loss: 13.3691 - accuracy: 0.4806 - auc: 0.7781
Epoch 5/50
18/18 [==============================] - 1s 50ms/step - loss: 10.2554 - accuracy: 0.4554 - auc: 0.7480
Epoch 6/50
18/18 [==============================] - 1s 50ms/step - loss: 4.6833 - accuracy: 0.4612 - auc: 0.8515
Epoch 7/50
18/18 [==============================] - 1s 50ms/step - loss: 9.0839 - accuracy: 0.4690 - auc: 0.7674
Epoch 8/50
18/18 [==============================] - 1s 51ms/step - loss: 6.4026 - accuracy: 0.4457 - auc: 0

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/tensor_util.py:436: ComplexWarning: Casting complex values to real discards the imaginary part
  nparray = values.astype(dtype.as_numpy_dtype)


------------------------------------------------------------------------
Training for fold 9 ...
Epoch 1/50
18/18 [==============================] - 9s 52ms/step - loss: 34.7225 - accuracy: 0.4729 - auc: 0.6248
Epoch 2/50
18/18 [==============================] - 1s 50ms/step - loss: 26.4791 - accuracy: 0.4419 - auc: 0.6598
Epoch 3/50
18/18 [==============================] - 1s 49ms/step - loss: 17.2579 - accuracy: 0.4554 - auc: 0.7111
Epoch 4/50
18/18 [==============================] - 1s 49ms/step - loss: 19.8289 - accuracy: 0.4787 - auc: 0.6195
Epoch 5/50
18/18 [==============================] - 1s 50ms/step - loss: 11.1062 - accuracy: 0.4612 - auc: 0.7393
Epoch 6/50
18/18 [==============================] - 1s 50ms/step - loss: 3.9575 - accuracy: 0.4050 - auc: 0.8808
Epoch 7/50
18/18 [==============================] - 1s 50ms/step - loss: 6.4934 - accuracy: 0.4399 - auc: 0.8117
Epoch 8/50
18/18 [==============================] - 1s 50ms/step - loss: 6.4906 - accuracy: 0.4360 - auc: 0

In [ ]:
test_probs.argmax(1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [ ]:
print(cv_accs)

[0.896551724137931, 0.8275862068965517, 0.8275862068965517, 0.8620689655172413, 0.8620689655172413, 0.8620689655172413, 0.8214285714285714, 0.8571428571428571, 0.9285714285714286, 0.8214285714285714]


In [ ]:
print(cv_aucs)

[0.9545454545454546, 0.9155844155844155, 0.95, 0.963768115942029, 0.9565217391304348, 0.6499999999999999, 0.9545454545454546, 0.8877551020408163, 0.859375, 0.9848484848484849]
